In [ ]:
from tensorflow.keras.preprocessing import image  
from tensorflow.keras.models import load_model  
import os  
import numpy as np  
import tensorflow as tf  
from tensorflow.keras.models import Model  
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.applications import EfficientNetV2B0 
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input  
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
import seaborn as sns  
import matplotlib.pyplot as plt  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
train_dir = '/content/drive/MyDrive/DatasetMRIAD/dataset_224_fix/train'
test_dir = '/content/drive/MyDrive/DatasetMRIAD/dataset_224_fix/test'
val_dir = '/content/drive/MyDrive/DatasetMRIAD/dataset_224_fix/val'

In [ ]:
import os
import matplotlib.pyplot as plt

base_dir = '/content/drive/MyDrive/DatasetMRIAD/dataset_224_fix/'
classes = ['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented', 'SevereDemented']

print("Train set:\n========================================")
for class_name in classes:
    num_images = len(os.listdir(os.path.join(base_dir, 'train', class_name)))
    print(f"{class_name}={num_images}")

print("Test set:\n========================================")
for class_name in classes:
    num_images = len(os.listdir(os.path.join(base_dir, 'test', class_name)))
    print(f"{class_name}={num_images}")

print("Validation set:\n========================================")
for class_name in classes:
    num_images = len(os.listdir(os.path.join(base_dir, 'val', class_name)))
    print(f"{class_name}={num_images}")

plt.figure(figsize=(10, 10))

for idx, class_name in enumerate(classes, start=1):
    class_dir = os.path.join(base_dir, 'test', class_name)
    class_images = os.listdir(class_dir)[:1]  # Take 1 example image from each class
    for i, image_name in enumerate(class_images, start=1):
        plt.subplot(4, 3, idx + i - 1)
        img = plt.imread(os.path.join(class_dir, image_name))
        plt.imshow(img, cmap='gray')
        plt.title(class_name)
        plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Specify the form of input
input_shape = (224, 224, 3)

from keras.applications import EfficientNetV2B0

base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=input_shape)


In [ ]:
# Insert the output of base_model into the variable x
x = base_model.output

# Insert the output of base_model into the variable x
x = GlobalAveragePooling2D()(x)

# Added Dense layer with 128 neurons and ReLU activation function
x = Dense(128, activation='relu')(x)

# Added Dropout layer to prevent overfitting
x = Dropout(0.5)(x)  # Dropout rate 0.5 means 50% of the neurons will be randomly deactivated

# Adding a Dense layer with 5 neurons (according to the number of classes) and softmax activation function
predictions = Dense(5, activation='softmax')(x)

# Form a model by connecting the inputs of the base_model to the outputs of the predictions layer
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False
# Compile model
sgd = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
# Define data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=input_shape[:2],
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=True)
val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=input_shape[:2],
                                                batch_size=32,
                                                class_mode='categorical',
                                                shuffle=False)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=input_shape[:2],
                                                  batch_size=32,
                                                  class_mode='categorical',
                                                  shuffle=False)

In [ ]:
model.summary()

In [ ]:
# Train the model with training data from train_generator
import time

# Start Time
start_time = time.time()

history = model.fit(
    train_generator,  # Generator for training data
    steps_per_epoch=len(train_generator),  # Number of steps per epoch, calculated from the generator length
    epochs=100,  # Number of epochs (complete iterations over the dataset)
    validation_data=val_generator, # Validation data used to evaluate model performance
    validation_steps=len(val_generator)  # Number of steps per epoch for validation, calculated from the generator length
)

end_time = time.time()

In [ ]:
# Calculate the time difference in seconds
training_time_seconds = end_time - start_time

# Convert to minutes
training_time_minutes = training_time_seconds / 60

print("Waktu pelatihan model:", training_time_minutes, "menit")

In [ ]:
# Save the model in .hdf5 format
model.save("EfficientNetV2B0.h5")

In [ ]:
# Retrieving training and validation accuracy values from history
acc = history.history['accuracy']  # Training accuracy at each epoch
val_acc = history.history['val_accuracy']  # Validation accuracy at each epoch

# Determine the number of epochs
epochs = range(len(acc))  # Number of epochs

# Plot the training and validation accuracy graphs against epochs
plt.plot(epochs, acc, 'r', label='Training accuracy')  
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')  
plt.title('Training and validation accuracy')  
plt.xlabel('Epoch')  
plt.ylabel('Accuracy')  
plt.legend() 
plt.figure()  


In [ ]:
# Take training and validation loss values from history
loss = history.history['loss'] 
val_loss = history.history['val_loss'] 

# Determine number of epochs 
epochs = range(len(loss)) 

# Plot the training and validation loss graphs against epochs
plt.plot(epochs, loss, 'r', label='Training loss')  
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss')  
plt.xlabel('Epoch')  
plt.ylabel('Loss') 
plt.legend()  
plt.figure()  

In [ ]:
# Evaluate the model using training data (train_generator) and print the training accuracy
evaluation = model.evaluate(train_generator) 
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%") 

# Evaluate the model using test data (test_generator) and print the test accuracy
evaluation = model.evaluate(test_generator) 
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%") 

# Evaluate the model using test data (test_generator) and print the test accuracy
evaluation = model.evaluate(val_generator)  
print(f"Validation Accuracy: {evaluation[1] * 100:.2f}%") 

In [ ]:
# Evaluate the model using the test generator data (test_generator)
model.evaluate(test_generator)

# Evaluate the model using validation generator data (test_generator)
model.evaluate(val_generator)

In [ ]:
preds = model.predict_generator(generator=test_generator) # Predicting class probabilities using test data generator
labels = 1 * (preds > 0.5)  # Converts the probability to a binary class (0 or 1) with a threshold of 0.5
# Retrieving accuracy and loss data from history
train_acc = history.history['accuracy'] 
val_acc = history.history['val_accuracy']  
train_loss = history.history['loss']  
val_loss = history.history['val_loss']

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# Using the model to make prediction on data test
test_generator.reset()
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
# Take the class with the highest probability value as the prediction
y_pred = np.argmax(pred, axis=1)
y_true = test_generator.classes

pd.DataFrame(classification_report(test_generator.classes, y_pred, output_dict=True))

In [ ]:
# Using the model to make prediction on data test
test_generator.reset()
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
y_pred = np.argmax(pred, axis=1)
y_true = test_generator.classes

# Create confusion matrix
cm = confusion_matrix(test_generator.classes, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming test_generator is already defined and configured

# Extract class labels directly from the test generator
# This assumes that the classes attribute is accessible and correctly ordered
class_labels = list(test_generator.class_indices.keys())

# Predict class probabilities using the test generator
test_generator.reset()
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)

# Convert probabilities to predicted class indices
y_pred = np.argmax(pred, axis=1)

# Actual class labels from the test generator
y_true = test_generator.classes

# Calculate overall accuracy
accuracy = accuracy_score(y_true, y_pred)
print('Overall Accuracy:', accuracy)

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix with class labels
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix with Class Labels')
plt.show()

# Generate and print classification report
report = classification_report(y_true, y_pred, target_names=class_labels)
print(report)


In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# Using the model to make predictions on test data
test_generator.reset()
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
# Take the class with the highest probability value as the prediction
y_pred = np.argmax(pred, axis=1)
y_true = test_generator.classes

# Calcukate overall accuracy
accuracy = accuracy_score(y_true, y_pred)
print('Overall Accuracy:', accuracy)

# Using the model to make predictions on test data
test_generator.reset()
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)

# Take the class with the highest probability value as the prediction
y_pred_classes = np.argmax(pred, axis=1)
y_true = test_generator.classes

# Measure akurasi
accuracy = accuracy_score(y_true, y_pred)
print('Accuracy:', accuracy)

# Measure presisi
precision = precision_score(y_true, y_pred, average='macro')
print('Precision:', precision)

# Measure recall
recall = recall_score(y_true, y_pred, average='macro')
print('Recall:', recall)

# Measure F1-score
f1 = f1_score(y_true, y_pred, average='macro')
print('F1-score:', f1)

# Measure confusion matrix
cm = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(cm)

In [ ]:
# Create plot learning curve
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.show()